In [76]:
import pickle
import pandas as pd
from tqdm import tqdm
# import numpy as np
import torch
from analysis import load_pckl
import warnings
import os
import glob2
warnings.filterwarnings('ignore')

In [77]:
def build_index(x):
    res = []
    ret = pd.DataFrame(columns=['Idx', 'Class', 'Path'])
    d = {}
    c = 0

    for idx, row in enumerate(x):

#         res.append( torch.load(row["Path"]+"/indiv_features.pt")[int(row["Idx"])].tolist() )
        path = row["Path"].replace("train_non_torch", "train-final")+"/indiv_features.pckl"
        if os.path.exists(path):
            with open(path, "rb") as f:
                data = pickle.load(f)
            if row["Idx"] < len(data):
                add = {'Idx':row['Idx'], 'Class':row['Class'], 'Path':row['Path'].replace("train_non_torch", "train-final")}
                if add in d.values():
                    continue
                res.append(data[row["Idx"]])
                d[c] = add
                if idx % 7500 == 0:
                    ret = pd.concat([ret, pd.DataFrame.from_dict(d, "index").drop_duplicates()])
                    d = {}
                c+= 1
        
    ret = pd.concat([ret, pd.DataFrame.from_dict(d, "index").drop_duplicates()])

    return np.array(res, dtype="float16"), ret 

In [9]:
# 1
mode = "train-final"
csv = pd.read_csv("data/{}/features.csv".format(mode),
                  names=["Idx", "Class", "Path"], low_memory=False)


In [13]:
idx = 0
to_rem = []
for row in tqdm(csv.to_dict('records')):
    try:
        int(row["Class"])
    except:
        csv.drop(index=idx, inplace=True)
    idx += 1

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2966024/2966024 [02:33<00:00, 19347.31it/s]


In [107]:
classes = sorted(csv["Class"].unique().tolist(), key=lambda x: int(x))
for cl in classes:
    print(cl, len(csv.loc[csv["Class"]==cl] ) )

2 29985
3 32605
4 767884
5 20020
6 632857
7 2455
8 800264
9 55656
10 4267
11 33387
12 872
13 29547
14 9769
15 28486
16 5741
17 4518
18 5752
19 131202
20 19196
21 301
22 5259
23 1325
24 95796
25 1190
26 1231
27 3888
28 5464
29 220
30 110
31 136
32 9169
33 400
34 25132
35 585
36 3617
37 58
38 14081
39 1482
40 627
41 38
42 446
43 40
44 30
45 9929
46 249
47 2691
48 49393
49 2268
50 476
51 276
52 28
53 215
54 368
55 2736
56 3118
57 125
58 7027
59 636
60 2219
61 93
62 99
63 206
64 13
65 2
66 12828
67 116
68 394
69 21
70 232
71 268
72 456
73 15
75 110
77 117
78 3099
79 36
80 289
81 2
82 85
84 28
86 8
88 7
89 12
90 312
93 1
94 8
95 43
96 203
97 303
98 2
99 8
100 26
101 4
106 155
107 100
108 26
111 2
113 14
114 235
116 3
118 576
120 46
121 5
122 8
123 245
124 8
125 269
127 2
129 2
131 87
132 3
133 103
138 3
139 313
144 2
146 76755
147 4
148 4
149 1


In [48]:
with open("data/train-final/88/47813/travel_website/6374438/indiv_features.pckl", "rb") as f:
        test = pickle.load(f)


In [111]:
for cl in tqdm(classes):
    same = csv.loc[csv["Class"] == cl].drop_duplicates().to_dict('records')
#     print(cl, len(same))
    res, index_csv = build_index(same)
#     new_df = pd.concat([new, index_csv])
    index_csv.to_csv('data/train-final/indices/{}.csv'.format(int(cl)), sep=",", columns=["Idx", "Class", "Path"], header=False, index=False, index_label=False)
    with open('data/train-final/indices/{}.pckl'.format(int(cl)), "wb") as pckl:
        pickle.dump(res, pckl) 


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [4:07:09<00:00, 124.62s/it]


In [11]:
test = []
with open("data/{}/features.csv".format("train-final")) as f, open("data/{}/features_new.csv".format("train-final"), 'w') as f2:
    for line in tqdm(f):
        if not line.strip() or line.strip().startswith(",") or not line.strip()[0].isdigit() or line.strip().isdecimal(): continue  # skip the empty line
        f2.write(line)  # non-empty line. Write it to output
        test.append(1)

2966024it [00:05, 550128.89it/s]


In [12]:
len(test)

2966024

In [19]:

to_be_rem = []
for idx, row in tqdm(enumerate(paths), total=len(paths)):
    try:
#         if not os.path.isfile(row+"/indiv_features.pt"): 
#         torch.load(row.replace("train_non_torch", "train-final")+"/indiv_features.pt")
        with open(row.replace("train_non_torch", "train-final")+"/indiv_features.pckl", "rb") as f:
            pickle.load(f)
#             to_be_rem.append(row)
    except:
        to_be_rem.append(row)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1025058/1025058 [1:19:44<00:00, 214.23it/s]


In [106]:
idx_csv

,Idx,Class,Path
0,1,22,data/train-final/6/17299/travel_website/6819299
1,1,22,data/train-final/6/49205/travel_website/6401997
2,3,22,data/train-final/59/15963/travel_website/6996893
3,2,22,data/train-final/59/26649/travel_website/7017328
4,1,22,data/train-final/59/22189/travel_website/6078338
...,...,...,...
3739,2,22,data/train-final/-1/204297/travel_website/9546730
3740,2,22,data/train-final/-1/24830/travel_website/7966266
3741,3,22,data/train-final/-1/3729/travel_website/7982609
3742,1,22,data/train-final/-1/81893/travel_website/6620444


In [112]:
for cl in tqdm(classes):
    idx_csv = pd.read_csv("data/train-final/indices/{}.csv".format(cl),
                      names=["Idx", "Class", "Path"], low_memory=False)

    with open("data/train-final/indices/{}.pckl".format(cl), "rb") as f:
        test = pickle.load(f)
        for idx, vec in enumerate(test):
            if not np.all( np.isfinite(vec) ) or np.any( np.isnan(vec) ):
                print("got one")
                test = np.delete(test, idx, 0)
                idx_csv.drop(index=idx, inplace=True)
    idx_csv.to_csv("data/train-final/indices/{}.csv".format(cl), sep=",", columns=["Idx", "Class", "Path"], header=False, index=False, index_label=False)
    with open("data/train-final/indices/{}.pckl".format(cl), "wb") as f:
        pickle.dump(test, f)

  8%|███████████▎                                                                                                                           | 10/119 [00:46<05:58,  3.29s/it]

got one


 11%|██████████████▋                                                                                                                        | 13/119 [00:46<02:53,  1.64s/it]

got one


 18%|███████████████████████▊                                                                                                               | 21/119 [00:50<00:54,  1.81it/s]

got one
got one


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 119/119 [00:56<00:00,  2.10it/s]


In [105]:
np.delete(test, 1, 0).shape

(3743, 2048)

In [21]:
csv = csv[~csv['Path'].isin(to_be_rem)]

In [18]:
# fix off by one error
csv["Class"]=csv.Class.astype(int)
csv["Class"] = csv.Class.apply(lambda x: x + 1)


In [19]:
csv.to_csv("data/train_non_torch/features.csv", sep=",", columns=["Idx", "Class", "Path"], header=False, index=False, index_label=False)

In [ ]:
tnt = build_index(csv.to_dict('records'))

 20%|████████████████████████████████████████████▋                                                                                                                                                                                | 599353/2965249 [44:43<1:39:20, 396.96it/s]

In [27]:
csv.to_dict('records')

[{'index': 0,
  'Idx': 0,
  'Class': 38,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007877'},
 {'index': 1,
  'Idx': 0,
  'Class': 4,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007872'},
 {'index': 2,
  'Idx': 1,
  'Class': 6,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007872'},
 {'index': 3,
  'Idx': 2,
  'Class': 8,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007872'},
 {'index': 4,
  'Idx': 0,
  'Class': 4,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007860'},
 {'index': 5,
  'Idx': 1,
  'Class': 19,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007860'},
 {'index': 6,
  'Idx': 2,
  'Class': 34,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007860'},
 {'index': 7,
  'Idx': 0,
  'Class': 4,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007861'},
 {'index': 8,
  'Idx': 1,
  'Class': 6,
  'Path': 'data/train_non_torch/6/30963/travel_website/6007861'},
 {'index': 9,
  'Idx': 2,
  'Class': 8,
  '

In [28]:
for i in tqdm(enumeate(csv.to_dict('records')),:
    pass

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2965249/2965249 [00:00<00:00, 6959967.53it/s]
